In [1]:
%load_ext autoreload
%autoreload 2
import os
import json
import sys

project_root = os.path.abspath(os.path.join(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

from transformers import AutoModel, AutoTokenizer

from data.utils import get_entity_idx, get_prompts

data_path = os.path.join(project_root, 'data/homograph_data/homograph_small.json')
with open(data_path) as f:
    data = json.load(f)  

In [3]:
model_ids = ["Qwen/Qwen3-4B"]#, "Qwen/Qwen3-8B", "Qwen/Qwen3-14B"]  # "Qwen/Qwen3-0.6B","Qwen/Qwen3-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_ids[0])

In [ ]:
prompts = get_prompts(data, context_type='minimal_context')
ent_idx = get_entity_idx(tokenizer, prompts)

[3,
 4,
 3,
 4,
 5,
 3,
 2,
 2,
 2,
 3,
 2,
 7,
 4,
 3,
 4,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 4,
 2,
 5,
 4,
 3,
 2,
 4,
 4,
 3,
 2,
 2,
 2,
 2,
 4,
 2,
 3,
 2,
 2,
 4,
 2,
 3,
 2,
 2,
 2]